In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
x = np.linspace(-10, 10, 10000) # start, finish, n points

def my_fxn(my_x):
    my_y = (my_x**2 - 1)/(my_x - 1)
    return my_y

In [9]:
# Using the delta method, find the slope of the tangent where x = -1
delta = 1e-06

x1 = -1
y1 = my_fxn(x1)

x2 = x1 + delta
y2 = my_fxn(x2)

# This 1.00000e-06 is practically zero.
m = (y2-y1) / (x2-x1)

m

1.000000000011061